In [ ]:
conda install -c conda-forge librosa

In [ ]:
import warnings                        # To ignore any warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%pylab inline
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import tensorflow as tf; print("Tensorflow version.... ",tf.__version__)
import keras ; print("Keras version... ", keras.__version__)

In [ ]:
INPUT_DIR = "/kaggle/input/heartbeat-sounds"

SAMPLE_RATE = 16000

MAX_SOUND_CLIP_DURATION = 12

In [ ]:
!pwd
!ls -all ../input

In [ ]:
set_a = pd.read_csv(INPUT_DIR+'/set_a.csv')
set_a.head()

In [ ]:
set_a_timing = pd.read_csv(INPUT_DIR+'/set_a_timing.csv')
set_a_timing.head()

In [ ]:
set_b = pd.read_csv(INPUT_DIR+'/set_b.csv')
set_b.head()

In [ ]:
frames = [set_a, set_b]
train_ab = pd.concat(frames)
train_ab

In [ ]:
nb_classes = train_ab.label.unique()

print("number of training examples : ", train_ab.shape[0]," Number of classes : ",len(nb_classes))
print(nb_classes)

*nan labeled examples are "unlabeled" test files*

In [ ]:
train_ab[train_ab.label == 'nan'].nunique()

In [ ]:
train_ab.label.value_counts()

In [ ]:
train_ab.groupby(['label','dataset']).count()

Visualize data distribution by class

In [ ]:
category_group = train_ab.groupby(['label','dataset']).count()
plot = category_group.unstack().reindex(category_group.unstack().sum(axis=1).sort_values().index)\
        .plot(kind='bar', stacked=False, title="Number of Audio Samples per Category", figsize=(16,5))

plot.set_xlabel('class')
plot.set_ylabel('samples count')


In [ ]:
print("minimum samples per class :", min(train_ab.label.value_counts()))
print("maximum samples per class :",max(train_ab.label.value_counts()))

# *Lets visit each class of labels one by one*

**a) NORMAL**

In [ ]:
normal_file = INPUT_DIR+'/set_a/normal__201105021654.wav'

In [ ]:
import IPython.display as ipd
ipd.Audio(normal_file)

In [ ]:
from scipy.io import wavfile
rate, signal = wavfile.read(normal_file)
print("Sampling Rate. ",rate)
print("Total samples  ",signal.shape[0])
print("Duration in seconds. ",signal.shape[0]/rate)
print(signal)

In [ ]:
plt.figure(figsize=(16,5))
plt.plot(signal, '-',)
plt.title('Normal')

Using **LIBROSA**

In [ ]:
signal1, rate1 = librosa.load(normal_file, duration=5)   #default sampling rate is 22 HZ
dur=librosa.get_duration(signal1)
print("Duration in seconds. ",librosa.get_duration(signal1))
print(signal1.shape, rate1)

In [ ]:
plt.figure(figsize=(16,3))
librosa.display.waveplot(signal1)
plt.title("Normal")

**b) Murmur**

In [ ]:
murmur_file = INPUT_DIR+'/set_a/murmur__201108222236.wav'
ipd.Audio(murmur_file)

In [ ]:
signal2, rate2 = librosa.load(murmur_file, duration=5)   #default sampling rate is 22 HZ
dur=librosa.get_duration(signal2)
print("Duration in seconds. ",librosa.get_duration(signal2))
print(signal2.shape, rate2)

In [ ]:
plt.figure(figsize=(16,3))
librosa.display.waveplot(signal2)
plt.title("murmur")

**c) Extrasystole**

In [ ]:
extrasystole_file = INPUT_DIR+'/set_b/extrastole__198_1308141739338_B1.wav'


In [ ]:
ipd.Audio(extrasystole_file)

In [ ]:
signal3, rate3 = librosa.load(extrasystole_file, duration=5)   #default sampling rate is 22 HZ
dur=librosa.get_duration(signal3)
print("Duration in seconds. ",librosa.get_duration(signal3))
print(signal3.shape, rate3)

In [ ]:
plt.figure(figsize=(16,5))
librosa.display.waveplot(signal3)
plt.title('Extrasystole')

**d) Artifact**

In [ ]:
artifact_file = INPUT_DIR+'/set_a/artifact__201012172012.wav'
ipd.Audio(artifact_file)

In [ ]:
signal4, rate4 = librosa.load(artifact_file, duration=5)   #default sampling rate is 22 HZ
dur=librosa.get_duration(signal4)
print("Duration in seconds. ",librosa.get_duration(signal4))
print(signal4.shape, rate4)

In [ ]:
plt.figure(figsize=(16,3))
librosa.display.waveplot(signal4)
plt.title("Artifact")

**e) Extra Heart Sound**

In [ ]:
extrahls_file = INPUT_DIR+'/set_a/extrahls__201101070953.wav'
ipd.Audio(extrahls_file)

In [ ]:
signal5, rate5 = librosa.load(extrahls_file, duration=5)
print("Duration ",librosa.get_duration(signal5))
print(signal5.shape, rate5)

In [ ]:
plt.figure(figsize=(16,3))
librosa.display.waveplot(signal5, sr = rate5)
plt.title("Extra Heart Sound")

In [ ]:
normal_file
signal, rate = librosa.load(normal_file)
mfcc = librosa.feature.mfcc(y=signal, sr = rate)
print(mfcc.shape)

In [ ]:
S = librosa.feature.melspectrogram(y=signal, sr=rate, n_mels=128,fmax=8000)
log_S=librosa.feature.mfcc(S=librosa.power_to_db(S))
print (log_S)

In [ ]:
mfcc = librosa.feature.mfcc(y = signal, sr = rate, n_mfcc = 40)

In [ ]:
print(mfcc.reshape([-1,1]).shape)

In [ ]:
40*345

# Visualizing MFCC series 

mel frequency cepstral coefficients (mfcc) which is by far the best way to numerically represent audio signal for ML related tasks

In [ ]:
plt.figure(figsize(12,3))
librosa.display.specshow(mfcc, x_axis='time')
plt.colorbar()
plt.title("MFCC")
plt.tight_layout()

In [ ]:
m_slaney = librosa.feature.mfcc(y=signal, sr=rate, dct_type=2)
plt.figure(figsize=(12,3))

m_htk = librosa.feature.mfcc(y=signal, sr=rate, dct_type=3)
plt.subplot(3,1,1)
librosa.display.specshow(m_slaney, x_axis='time')
plt.title("dct_type = 2")
plt.colorbar()
plt.subplot(3,1,3)
librosa.display.specshow(m_htk, x_axis='time')
plt.title("dct_type = 3")
plt.colorbar()

**LOADING DATA**

In [ ]:
train_ab

In [ ]:
print("Number of training examples : ",train_ab.shape[0], " Number of classes : ", train_ab.label.nunique())

In [ ]:
def audio_norm(data):
    max_data = max(data)
    min_data = min(data)
    data = (data-min_data)/(max_data-min_data+0.0001)
    return data-0.5

In [ ]:
def load_file_data (folder_name,file_names, duration=12, sr=16000):
    input_length=sr*duration
    # function to load files and extract features
    # file_names = glob.glob(os.path.join(folder, '*.wav'))
    data = []
    for file_name in file_names:
        try:
            sound_file=folder_name+file_name
            print ("load file ",sound_file)
            # use kaiser_fast technique for faster extraction
            X, sr = librosa.load( sound_file, sr=sr, duration=duration,res_type='kaiser_fast') 
            dur = librosa.get_duration(y=X, sr=sr)
            # pad audio file same duration
            if (round(dur) < duration):
                print ("fixing audio lenght :", file_name)
                y = librosa.util.fix_length(X, input_length)                
            #normalized raw audio 
            X = audio_norm(X)            
            # extract normalized mfcc feature from data
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40).T,axis=0)             
        except Exception as e:
            print("Error encountered while parsing file: ", file)        
        feature = np.array(mfccs).reshape([-1,1])
        data.append(feature)
    return data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

CLASSES = ['artifact','murmur','normal']
NB_CLASSES = len(CLASSES)

label_to_int = {k:v for v,k in enumerate(CLASSES)}
print(label_to_int)
print(" ")

int_to_label = {v:k for v,k in enumerate(CLASSES)}
print(int_to_label)

Loading dataset a

In [ ]:
import os, fnmatch

A_folder = INPUT_DIR+'/set_a/'

A_artifact_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a/'),'artifact*.wav')
A_artifact_sounds = load_file_data(folder_name=A_folder, file_names=A_artifact_files, duration=MAX_SOUND_CLIP_DURATION)
A_artifact_labels = [0 for items in A_artifact_files]

A_normal_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a/'), 'normal*.wav')
A_normal_sounds = load_file_data(folder_name = A_folder, file_names = A_normal_files, duration=MAX_SOUND_CLIP_DURATION)
A_normal_labels = [2 for items in A_normal_files]

A_extrahls_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a/'), 'extrahls*.wav')
A_extrahls_sounds = load_file_data(folder_name=A_folder, file_names=A_extrahls_files, duration=MAX_SOUND_CLIP_DURATION)
A_extrahls_labels = [1 for items in A_extrahls_files]

A_murmur_files= fnmatch.filter(os.listdir(INPUT_DIR+'/set_a/'), 'murmur*.wav')
A_murmur_sounds = load_file_data(folder_name=A_folder, file_names=A_murmur_files, duration=MAX_SOUND_CLIP_DURATION)
A_murmur_labels = [1 for items in A_murmur_files]

A_unlabelledtest_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_a/'), 'Aunlabelledtest*.wav')
A_unlabelledtest_sounds = load_file_data(folder_name=A_folder, file_names=A_unlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION)
A_unlabelledtest_labels = [-1 for items in A_unlabelledtest_files]

print("loaded dataset-a")

Loading Dataset-b

In [ ]:


B_folder = INPUT_DIR+'/set_b/'


B_normal_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b/'), 'normal*.wav')
B_normal_sounds = load_file_data(folder_name = B_folder, file_names = B_normal_files, duration=MAX_SOUND_CLIP_DURATION)
B_normal_labels = [2 for items in B_normal_files]

B_extrastole_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b/'), 'extrastole*.wav')
B_extrastole_sounds = load_file_data(folder_name=B_folder, file_names=B_extrastole_files, duration=MAX_SOUND_CLIP_DURATION)
B_extrastole_labels = [1 for items in B_extrastole_files]

B_murmur_files= fnmatch.filter(os.listdir(INPUT_DIR+'/set_b/'), 'murmur*.wav')
B_murmur_sounds = load_file_data(folder_name=B_folder, file_names=B_murmur_files, duration=MAX_SOUND_CLIP_DURATION)
B_murmur_labels = [1 for items in B_murmur_files]

B_unlabelledtest_files = fnmatch.filter(os.listdir(INPUT_DIR+'/set_b/'), 'Bunlabelledtest*.wav')
B_unlabelledtest_sounds = load_file_data(folder_name=B_folder, file_names= B_unlabelledtest_files, duration=MAX_SOUND_CLIP_DURATION)
B_unlabelledtest_labels = [-1 for items in B_unlabelledtest_files]

print("loaded dataset-b")

Combining set-a   set-b

In [ ]:
x_data = np.concatenate((A_artifact_sounds, A_normal_sounds, A_extrahls_sounds, A_murmur_sounds,
                        B_normal_sounds,B_murmur_sounds, B_extrastole_sounds))
y_data = np.concatenate((A_artifact_labels, A_normal_labels, A_extrahls_labels, A_murmur_labels,
                        B_normal_labels,B_murmur_labels, B_extrastole_labels))

x_unlabelled = np.concatenate((A_unlabelledtest_sounds, B_unlabelledtest_sounds))
y_unlabelled = np.concatenate((A_unlabelledtest_labels, B_unlabelledtest_labels))

print("Combined data size : ",len(x_data), " and unlabelled ",len(x_unlabelled))

In [ ]:
seed = 1000

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, train_size=0.8,random_state=seed, shuffle=True)
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.9, random_state = seed, shuffle=True)

y_train = np.array(keras.utils.to_categorical(y_train, len(CLASSES)))
# y_test = np.array(keras.utils.to_categorical(y_test, len(CLASSES)))
y_val = np.array(keras.utils.to_categorical(y_val, len(CLASSES)))
y_unlabelled = np.array(keras.utils.to_categorical(y_unlabelled, len(CLASSES)))

In [ ]:
y_unlabelled.shape

In [ ]:
print ("label shape: ", y_data.shape)
print ("data size of the array: : %s" % y_data.size)
print ("length of one array element in bytes: ", y_data.itemsize)
print ("total bytes consumed by the elements of the array: ", y_data.nbytes)
print (y_data[1])
print ("")
print ("audio data shape: ", x_data.shape)
print ("data size of the array: : %s" % x_data.size)
print ("length of one array element in bytes: ", x_data.itemsize)
print ("total bytes consumed by the elements of the array: ", x_data.nbytes)
#print (x_data[1])
print ("")
print ("training data shape: ", x_train.shape)
print ("training label shape: ", y_train.shape)
print ("")
print ("validation data shape: ", x_val.shape)
print ("validation label shape: ", y_val.shape)
# print ("")
# print ("test data shape: ", x_test.shape)
# print ("test label shape: ", y_test.shape)
print("")
print ("unlabelled data shape: ", x_unlabelled.shape)
print ("unlabelled label shape: ", y_unlabelled.shape)

**BUILDING MODEL**

In [ ]:
pip install livelossplot

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,ProgbarLogger
from keras.utils import np_utils
from sklearn import metrics 
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import itertools
from keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

from livelossplot import PlotLossesKeras

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(units=64, dropout=0.2, return_sequences=True), input_shape=(40,1)))
model.add(Bidirectional(LSTM(units=32, dropout=0.2, return_sequences=False)))
model.add(Dense(len(CLASSES), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', 
             metrics=['acc'])
model.summary()

In [ ]:
max_patience = 12
max_epochs = 100
max_batch = 32

early_stopping = EarlyStopping(monitor='val_accuracy', patience = max_patience, verbose=0, mode='max', restore_best_weights=False)

callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.00001),
          early_stopping,
         PlotLossesKeras()]

print("training started..")
history = model.fit(x_train, y_train,
                   batch_size=max_batch,
                   epochs=max_epochs,
                   verbose=1,
                   validation_data=(x_val, y_val),
                   callbacks=callbacks)

Model Evaluation

In [ ]:
score = model.evaluate(x_train, y_train, verbose=0) 
print ("model train data score       : ",round(score[1]*100) , "%")

# score = model.evaluate(x_test, y_test, verbose=0) 
# print ("model test{split} data score : ",round(score[1]*100) , "%")

score = model.evaluate(x_val, y_val, verbose=0) 
print ("model validation data score  : ", round(score[1]*100), "%")

score = model.evaluate(x_unlabelled, y_unlabelled, verbose=0) 
print ("model unlabeled data score   : ", round(score[1]*100), "%")